In [ ]:
#This needs to be run on the instance where PostGIS is also running with SedonaDB installed
import geopandas as gpd
import pandas as pd
import threading
from sqlalchemy import create_engine
import sedona.db
import yaml
from time import perf_counter
from pyarrow import parquet as pq
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import random
sd = sedona.db.connect()

# Create a PostGIS connection (adjust parameters as needed)
user="postgres"
password="test"
host="localhost"
engine = create_engine("postgresql://"+user+":"+password+"@"+host+":5432/postgres")
#set size to the desired dataset size
size= 100000000

#by default, the benchmark runs single-threaded, to run multi-threaded, change num_threads to the desired number of threads
num_threads = 1




In [2]:
import pyarrow as pa
import pyarrow.parquet as pq
import psycopg2

conn = psycopg2.connect("dbname=postgres user=postgres password=test host=localhost")
cur = conn.cursor(name="stream_export")   # server-side cursor

cur.execute("""
    SELECT 
        flightid,
        airplanetype,
        origin,
        destination,
        ST_AsText(geom) AS geom,
        timestamp,
        altitude 
        FROM flight_points
""")
chunk_size = 500_000
writer = None

while True:
    rows = cur.fetchmany(chunk_size)
    if not rows:
        break
    
    table = pa.Table.from_pylist([dict(zip([desc[0] for desc in cur.description], row)) for row in rows])
    
    if writer is None:
        writer = pq.ParquetWriter("output.parquet", table.schema, compression="snappy")
    
    writer.write_table(table)

if writer:
    writer.close()

cur.close()
conn.close()


In [ ]:
chunks = []
chunk_size = 1000000

sql = f"SELECT * FROM ride_points LIMIT {size}"
for chunk in gpd.read_postgis(sql, engine, geom_col="geom", chunksize=chunk_size):
    chunks.append(chunk)
    print(f"Loaded chunk with {len(chunk)} rows")

gdf = gpd.GeoDataFrame(pd.concat(chunks, ignore_index=True))
print("Final combined GeoDataFrame:", len(gdf))

In [3]:
#cycling

gdf = gpd.read_parquet("ride_points.parquet")

gdf2 = gpd.read_postgis(
    "SELECT * FROM universities",
    con=engine,
    geom_col="geom"
)
gdf3 = gpd.read_postgis(
    "SELECT * FROM berlin_districts",
    con=engine,
    geom_col="geom"
)

gdf4 = gpd.read_postgis(
    "SELECT ride_id, rider_id, origin_geom, start_time FROM ride_trajectories",
    con=engine,
    geom_col="origin_geom"
)

gdf5 = gpd.read_postgis(
    "SELECT ride_id, rider_id, destination_geom, end_time FROM ride_trajectories",
    con=engine,
    geom_col="destination_geom"
)

df = sd.create_data_frame(gdf)
df2 = sd.create_data_frame(gdf2)
df3 = sd.create_data_frame(gdf3)
df4 = sd.create_data_frame(gdf4)
df5 = sd.create_data_frame(gdf5)

df.to_view("ride_points", overwrite=True)
df2.to_view("universities", overwrite=True)
df3.to_view("berlin_districts", overwrite=True)
df4.to_view("ride_trajectories_start", overwrite=True)
df5.to_view("ride_trajectories_end", overwrite=True)



In [ ]:
#save gdf to parquet files
gdf.to_parquet("ride_points.parquet")
gdf2.to_parquet("universities.parquet")
gdf3.to_parquet("berlin_districts.parquet")
gdf4.to_parquet("ride_trajectories_start.parquet")
gdf5.to_parquet("ride_trajectories_end.parquet")

In [ ]:

sd.read_parquet("ride_points.parquet").to_view("ride_points", overwrite=True)
sd.read_parquet("universities.parquet").to_view("universities", overwrite=True)
sd.read_parquet("berlin_districts.parquet").to_view("berlin_districts", overwrite=True)
sd.read_parquet("ride_trajectories_start.parquet").to_view("ride_trajectories_start", overwrite=True)
sd.read_parquet("ride_trajectories_end.parquet").to_view("ride_trajectories_end", overwrite=True)






FileNotFoundError: [Errno 2] Failed to open local file 'ride_trajectories_start.parquet'. Detail: [errno 2] No such file or directory

In [2]:
chunks = []
chunk_size = 1000000

sql = f"SELECT * FROM crossing_points LIMIT {size}"
counter = 0
for chunk in gpd.read_postgis(sql, engine, geom_col="geom", chunksize=chunk_size):
    chunks.append(chunk)
    #write chunks to parquet if chunk count exceeds 10 million 
    if len(chunks) * chunk_size >= 10000000:
        gdf_chunk = gpd.GeoDataFrame(pd.concat(chunks, ignore_index=True))
        gdf_chunk.to_parquet(f"crossing_points_part_{counter}.parquet")
        counter += 1
        print(f"Wrote crossing_points_part_{counter}.parquet with {len(gdf_chunk)} rows")
        chunks = []
if len(chunks) > 0:
    gdf_chunk = gpd.GeoDataFrame(pd.concat(chunks, ignore_index=True))
    gdf_chunk.to_parquet(f"crossing_points_part_{counter}.parquet")
else:
    counter -= 1
files = [f"crossing_points_part_{i}.parquet" for i in range(counter + 1)]
schema = pq.ParquetFile(files[0]).schema_arrow
with pq.ParquetWriter("crossing_points.parquet", schema=schema) as writer:
    for file in files:
        writer.write_table(pq.read_table(file, schema=schema))

Wrote crossing_points_part_1.parquet with 10000000 rows
Wrote crossing_points_part_2.parquet with 10000000 rows
Wrote crossing_points_part_3.parquet with 10000000 rows
Wrote crossing_points_part_4.parquet with 10000000 rows
Wrote crossing_points_part_5.parquet with 10000000 rows
Wrote crossing_points_part_6.parquet with 10000000 rows
Wrote crossing_points_part_7.parquet with 10000000 rows
Wrote crossing_points_part_8.parquet with 10000000 rows
Wrote crossing_points_part_9.parquet with 10000000 rows


In [2]:


gdf = gpd.read_parquet("crossing_points.parquet")

gdf2 = gpd.read_postgis(
    "SELECT * FROM harbours",
    con=engine,
    geom_col="geom"
)
gdf3 = gpd.read_postgis(
    "SELECT * FROM islands",
    con=engine,
    geom_col="geom"
)

gdf4= gpd.read_postgis(
    "SELECT crossing_id, origin_geom, start_time FROM crossing_trajectories",
    con=engine,
    geom_col="origin_geom"
)
gdf5= gpd.read_postgis(
    "SELECT crossing_id, destination_geom, end_time FROM crossing_trajectories",
    con=engine,
    geom_col="destination_geom"
)


# Create SedonaDB DataFrames
df = sd.create_data_frame(gdf)
print("Successfully created Sedona DataFrame for crossing points")
df2 = sd.create_data_frame(gdf2)
df3 = sd.create_data_frame(gdf3)
df4 = sd.create_data_frame(gdf4)
df5 = sd.create_data_frame(gdf5)

# Register the raw data as temporary views
df.to_view("crossing_points", overwrite=True)
print("Successfully registered crossing points view")
df2.to_view("harbours", overwrite=True)
df3.to_view("islands", overwrite=True)
df4.to_view("crossing_trajectories_start", overwrite=True)
df5.to_view("crossing_trajectories_end", overwrite=True)

Successfully created Sedona DataFrame for crossing points
Successfully registered crossing points view


In [3]:


gdf.to_parquet("crossing_points.parquet")
gdf2.to_parquet("harbours.parquet")
gdf3.to_parquet("islands.parquet") 
gdf4.to_parquet("crossing_trajectories_start.parquet")
gdf5.to_parquet("crossing_trajectories_end.parquet")

In [ ]:
sd.read_parquet("crossing_points.parquet").to_view("crossing_points", overwrite=True)
sd.read_parquet("harbours.parquet").to_view("harbours", overwrite=True)
sd.read_parquet("islands.parquet").to_view("islands", overwrite=True)
sd.read_parquet("crossing_trajectories_start.parquet").to_view("crossing_trajectories_start", overwrite=True)
sd.read_parquet("crossing_trajectories_end.parquet").to_view("crossing_trajectories_end", overwrite=True)



FileNotFoundError: [Errno 2] Failed to open local file 'crossing_trajectories_start.parquet'. Detail: [errno 2] No such file or directory

In [3]:
# aviation
chunks = []
chunk_size = 1000000

sql = f"SELECT * FROM flight_points LIMIT {size}"
counter = 0
for chunk in gpd.read_postgis(sql, engine, geom_col="geom", chunksize=chunk_size):
    chunks.append(chunk)
    #write chunks to parquet if chunk count exceeds 10 million 
    if len(chunks) * chunk_size >= 10000000:
        gdf_chunk = gpd.GeoDataFrame(pd.concat(chunks, ignore_index=True))
        gdf_chunk.to_parquet(f"flight_points{counter}.parquet")
        counter += 1
        print(f"Wrote f{counter}.parquet with {len(gdf_chunk)} rows")
        chunks = []
if len(chunks) > 0:
    gdf_chunk = gpd.GeoDataFrame(pd.concat(chunks, ignore_index=True))
    gdf_chunk.to_parquet(f"flight_points_part_{counter}.parquet")
else:
    counter -= 1
counter = 25
files = [f"flight_points{i}.parquet" for i in range(counter + 1)]
schema = pq.ParquetFile(files[0]).schema_arrow
with pq.ParquetWriter("flight_points.parquet", schema=schema) as writer:
    for file in files:
        writer.write_table(pq.read_table(file, schema=schema))


: 

In [ ]:

gdf = gpd.read_parquet("flight_points.parquet")

gdf2 = gpd.read_postgis(
    "SELECT name, ST_SetSRID(geom,3857) as geom FROM districts",
    con=engine,
    geom_col="geom"
)
gdf3 = gpd.read_postgis(
    "SELECT name, ST_SetSRID(geom,3857) as geom FROM municipalities",
    con=engine,
    geom_col="geom"
)
gdf4 = gpd.read_postgis(
    "SELECT name, ST_SetSRID(geom,3857) as geom FROM cities",
    con=engine,
    geom_col="geom"
)
gdf5 = gpd.read_postgis(
    "SELECT name, ST_SetSRID(geom,3857) as geom FROM counties",
    con=engine,
    geom_col="geom"
)
gdf6 = gpd.read_postgis(
    "SELECT flightid, origin, origin_geom, start_time FROM flight_trajectories",
    con=engine,
    geom_col="origin_geom"
)
gdf7 = gpd.read_postgis(
    "SELECT flightid, destination, destination_geom, end_time FROM flight_trajectories",
    con=engine,
    geom_col="destination_geom"
)
df = sd.create_data_frame(gdf)
df2 = sd.create_data_frame(gdf2)
df3 = sd.create_data_frame(gdf3)
df4 = sd.create_data_frame(gdf4)
df5 = sd.create_data_frame(gdf5)
df6 = sd.create_data_frame(gdf6)
df7 = sd.create_data_frame(gdf7)

df.to_view("flight_points", overwrite=True)
df2.to_view("districts", overwrite=True)
df3.to_view("municipalities", overwrite=True)
df4.to_view("cities", overwrite=True)
df5.to_view("counties", overwrite=True)
df6.to_view("flight_trajectories_start", overwrite=True)
df7.to_view("flight_trajectories_end", overwrite=True)



In [3]:
#save to parquet
gdf.to_parquet("flight_points.parquet")
gdf2.to_parquet("districts.parquet")
gdf3.to_parquet("municipalities.parquet")
gdf4.to_parquet("cities.parquet")
gdf5.to_parquet("counties.parquet")
gdf6.to_parquet("flight_trajectories_start.parquet")
gdf7.to_parquet("flight_trajectories_end.parquet")


NameError: name 'gdf' is not defined

In [3]:
sd.read_parquet("output.parquet").to_view("flight_points_temp", overwrite=True)

#drop the flight_points table if exists
sd.sql("DROP TABLE IF EXISTS flight_points;")

#change the geom column to an actual geom with SRID 3857
sd.sql("""
CREATE TABLE flight_points AS
SELECT 
    flightid, 
    airplanetype, 
    origin, 
    destination,
    ST_SetSRID(ST_GeomFromWKT(geom), 3857) AS geom,
    timestamp,
    altitude
FROM flight_points_temp;
""")

sd.sql("DROP VIEW IF EXISTS flight_points_temp;")

In [4]:

sd.read_parquet("districts.parquet").to_view("districts", overwrite=True)
sd.read_parquet("municipalities.parquet").to_view("municipalities", overwrite=True)
sd.read_parquet("cities.parquet").to_view("cities", overwrite=True)
sd.read_parquet("counties.parquet").to_view("counties", overwrite=True)
sd.read_parquet("flight_trajectories_start.parquet").to_view("flight_trajectories_start", overwrite=True)
sd.read_parquet("flight_trajectories_end.parquet").to_view("flight_trajectories_end", overwrite=True)


In [ ]:
#generate random 5 character string for benchmark experiment id
#however, if you want to analzye it together with the other systems, use the same id as before (i.e, change it to the id of the other database systems after execution)
#threaded execution can be found below, functional but currently commented out

id = ''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789', k=5))
logging_path = f"/home/tim/client/benchmark_client/SedonaDBquery_results_{id}.log"

#Specify the benchmark to run here, using either ais, aviation, or cycling
benchmark = "aviation"

#specify the user
user="tim"

config_path  = f"/home/{user}/client/benchmark_client/src/main/resources/{benchmark}/sedonaSQL_queries.yaml"
config = None
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

#--single--

results = []

for q in config:
    name = q['name']
    sql = q['sql']
    print(name)
    print(sql)
    start = perf_counter()
    result = sd.sql(sql)
    result_count = result.count()
    end = perf_counter()

    result.show()
    print(name)
    duration = end - start

    results.append((name, result_count, duration))

with open(logging_path, 'w+') as log_file:
    for result in results:
        log_file.write(
            f"[SedonaDB] [{benchmark}] query: {result[0]}: {result[2] * 1000}ms, result count: {result[1]}\n"
        )

#--end-single--
#this part is used for threaded execution and parallel querying, to use it, comment out the part from single to end-single and uncomment below

# def run_query(name, sql):
#     """Execute a single query and return timing + partial results."""
#     thread_name = threading.current_thread().name
#     print(name)
#     start = perf_counter()
#     result = sd.sql(sql)
#     result_count = result.count()
#     end = perf_counter()
#     result.show()
#     print(name)
#     duration = end - start
#     return thread_name, name, result_count, duration

# # Run in parallel
# results = []
# with ThreadPoolExecutor(max_workers=num_threads) as executor:
#     for q in config:
#         future_to_query = executor.submit(run_query, q['name'], q['sql'])
#     for future in as_completed(future_to_query):
#         thread_name, query, count, duration = future.result()
#         #log to file

#         results.append((query, count, duration))
# #create log file if not exists, and append results

# with open(logging_path, 'w+') as log_file:
#     for result in results:
#         log_file.write(f"[SedonaDB] [{benchmark}] query: {result[0]}: {result[2]*1000}ms, result count: {result[1]}\n") 


┌──────────────┬──────────────────────────────┬─────────────┐
│  column_name ┆           data_type          ┆ is_nullable │
│     utf8     ┆             utf8             ┆     utf8    │
╞══════════════╪══════════════════════════════╪═════════════╡
│ flightid     ┆ Int64                        ┆ YES         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ airplanetype ┆ Utf8View                     ┆ YES         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ origin       ┆ Utf8View                     ┆ YES         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ destination  ┆ Utf8View                     ┆ YES         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ geom         ┆ Binary                       ┆ YES         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ timestamp    ┆ Timestamp(Microsecond, None) ┆ YES         │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ altitu